# Lance sift example

* Download ANN_SIFT1M from: http://corpus-texmex.irisa.fr/ and store it in your `~/data` directory.

In [1]:
!tar -xzf ~/data/sift.tar.gz

In [1]:
import shutil

import duckdb
import lance
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset

In [6]:
import struct

from lance.vector import vec_to_table

uri = "vec_data.lance"

with open("sift/sift_base.fvecs", mode="rb") as fobj:
    buf = fobj.read()
    data = np.array(
        struct.unpack("<128000000f", buf[4 : 4 + 4 * 1000000 * 128])
    ).reshape((1000000, 128))
    dd = dict(zip(range(1000000), data))

table = vec_to_table(dd)
lance.write_dataset(table, uri, max_rows_per_group=8192, max_rows_per_file=1024 * 1024)

/Users/matthew.powers/opt/miniconda3/envs/lance-096/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
uri = "vec_data.lance"
sift1m = lance.dataset(uri)

In [3]:
samples = duckdb.query("SELECT vector FROM sift1m USING SAMPLE 100").to_df().vector
samples

0     [40.0, 23.0, 2.0, 0.0, 6.0, 6.0, 10.0, 21.0, 1...
1     [11.0, 101.0, 22.0, 38.0, 52.0, 3.0, 1.0, 13.0...
2     [23.0, 0.0, 0.0, 28.0, 128.0, 114.0, 28.0, 5.0...
3     [34.0, 99.0, 15.0, 0.0, 1.793662034335766e-43,...
4     [12.0, 14.0, 101.0, 1.793662034335766e-43, 68....
                            ...                        
95    [18.0, 110.0, 3.0, 0.0, 4.0, 36.0, 26.0, 10.0,...
96    [25.0, 4.0, 9.0, 23.0, 8.0, 16.0, 81.0, 119.0,...
97    [0.0, 34.0, 36.0, 24.0, 22.0, 1.0, 2.0, 2.0, 1...
98    [0.0, 13.0, 11.0, 4.0, 17.0, 82.0, 46.0, 5.0, ...
99    [1.793662034335766e-43, 50.0, 12.0, 3.0, 8.0, ...
Name: vector, Length: 100, dtype: object

In [4]:
import time

start = time.time()
tbl = sift1m.to_table(
    columns=["id"], nearest={"column": "vector", "q": samples[0], "k": 10}
)
end = time.time()

In [5]:
print(f"Time(sec): {end-start}")

Time(sec): 0.01596212387084961


In [6]:
print(tbl.to_pandas())

       id                                             vector     _distance
0  526739  [40.0, 23.0, 2.0, 0.0, 6.0, 6.0, 10.0, 21.0, 1...  10692.439453
1  837113  [0.0, 7.0, 22.0, 5.0, 7.0, 14.0, 5.0, 0.0, 0.0...  45813.679688
2   16544  [38.0, 1.0, 2.0, 12.0, 7.0, 17.0, 19.0, 27.0, ...  52832.359375
3  267836  [31.0, 67.0, 28.0, 10.0, 0.0, 0.0, 0.0, 0.0, 2...  57222.203125
4  875684  [7.0, 19.0, 23.0, 22.0, 15.0, 5.0, 3.0, 11.0, ...  59004.722656
5  248486  [0.0, 19.0, 22.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.0...  60655.894531
6  462497  [5.0, 17.0, 42.0, 57.0, 2.0, 0.0, 0.0, 33.0, 4...  60739.160156
7  947150  [66.0, 14.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 28....  61110.132812
8  874394  [18.0, 46.0, 30.0, 0.0, 0.0, 0.0, 0.0, 5.0, 20...  61147.718750
9  548411  [0.0, 2.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  63139.500000


In [12]:
%%time

sift1m.create_index(
    "vector", index_type="IVF_PQ", num_partitions=256, num_sub_vectors=16
)

CPU times: user 1min 28s, sys: 2.82 s, total: 1min 31s
Wall time: 11.2 s


In [13]:
sift1m = lance.dataset(uri)

In [14]:
import time

tot = 0
for q in samples:
    start = time.time()
    tbl = sift1m.to_table(nearest={"column": "vector", "q": q, "k": 10})
    end = time.time()
    tot += end - start

print(f"Avg(sec): {tot / len(samples)}")
print(tbl.to_pandas())

Avg(sec): 0.0010939693450927733
       id                                             vector     _distance
0     956  [39.0, 20.0, 0.0, 2.0, 36.0, 118.0, 90.0, 32.0...  16214.018555
1  886799  [128.0, 31.0, 1.0, 0.0, 121.0, 128.0, 86.0, 1....  69260.789062
2  694718  [53.0, 0.0, 1.0, 0.0, 25.0, 122.0, 116.0, 26.0...  71173.421875
3  685559  [54.0, 6.0, 1.0, 0.0, 58.0, 133.0, 122.0, 8.0,...  75531.687500
4  847196  [65.0, 21.0, 0.0, 1.0, 116.0, 70.0, 81.0, 28.0...  76660.921875
5  282692  [112.0, 30.0, 9.0, 1.0, 97.0, 84.0, 84.0, 8.0,...  78626.078125
6  939173  [28.0, 35.0, 1.0, 0.0, 105.0, 120.0, 57.0, 11....  78680.429688
7  109316  [22.0, 10.0, 19.0, 37.0, 44.0, 61.0, 82.0, 31....  78744.492188
8  112670  [62.0, 14.0, 1.0, 1.0, 73.0, 123.0, 123.0, 20....  78891.125000
9   97706  [6.0, 16.0, 12.0, 0.0, 119.0, 67.0, 32.0, 9.0,...  79190.671875


In [15]:
sift1m.to_table(
    nearest={
        "column": "vector",
        "q": samples[0],
        "k": 10,
        "nprobes": 10,
        "refine_factor": 5,
    }
).to_pandas()

,id,vector,_distance
0,695882,"[51.0, 5.0, 15.0, 27.0, 16.0, 14.0, 89.0, 6.0,...",0.0
1,698462,"[57.0, 2.0, 4.0, 13.0, 16.0, 23.0, 112.0, 15.0...",75052.0
2,572171,"[22.0, 2.0, 1.0, 1.0, 21.0, 63.0, 125.0, 2.0, ...",80446.0
3,264119,"[5.0, 1.0, 0.0, 0.0, 6.0, 18.0, 67.0, 7.0, 1.0...",81585.0
4,462263,"[40.0, 9.0, 1.0, 18.0, 76.0, 19.0, 64.0, 0.0, ...",81728.0
5,268247,"[2.0, 0.0, 3.0, 11.0, 51.0, 32.0, 121.0, 15.0,...",82007.0
6,353516,"[11.0, 1.0, 0.0, 0.0, 11.0, 81.0, 137.0, 15.0,...",83446.0
7,811982,"[9.0, 3.0, 2.0, 0.0, 12.0, 33.0, 125.0, 8.0, 0...",84208.0
8,692270,"[54.0, 0.0, 1.0, 3.0, 62.0, 84.0, 85.0, 8.0, 1...",85731.0
9,29726,"[88.0, 52.0, 21.0, 10.0, 7.0, 12.0, 41.0, 26.0...",88305.0


In [7]:
tbl = sift1m.to_table()
tbl = tbl.append_column("item_id", pa.array(range(len(tbl))))
tbl = tbl.append_column("revenue", pa.array((np.random.randn(len(tbl))+5)*1000))
tbl.to_pandas()

,id,vector,item_id,revenue
0,0,"[0.0, 16.0, 35.0, 5.0, 32.0, 31.0, 14.0, 10.0,...",0,3907.585076
1,1,"[1.8e-43, 14.0, 35.0, 19.0, 20.0, 3.0, 1.0, 13...",1,5342.165841
2,2,"[33.0, 1.8e-43, 0.0, 1.0, 5.0, 3.0, 44.0, 40.0...",2,5436.595909
3,3,"[23.0, 10.0, 1.8e-43, 12.0, 47.0, 14.0, 25.0, ...",3,5038.238291
4,4,"[27.0, 29.0, 21.0, 1.8e-43, 1.0, 1.0, 0.0, 0.0...",4,5165.215941
...,...,...,...,...
999995,999995,"[8.0, 9.0, 5.0, 0.0, 10.0, 39.0, 72.0, 68.0, 3...",999995,4792.169119
999996,999996,"[3.0, 28.0, 55.0, 29.0, 35.0, 12.0, 1.0, 2.0, ...",999996,4561.606815
999997,999997,"[0.0, 13.0, 41.0, 72.0, 40.0, 9.0, 0.0, 0.0, 0...",999997,4195.688559
999998,999998,"[41.0, 121.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24...",999998,3317.930871
